In [ ]:
# 本篇介绍类SummaryWriter
# class torch.utils.tensorboard.writer.SummaryWriter
# 用于生成并写入tensorboard记录到磁盘中

# 方法:
# add_scalar
# add_scalars
# add_image
# add_images
# add_figure
# add_video
# add_audio
# add_text
# add_graph
# add_embedding
# add_pr_curve
# add_custom_scalars
# add_mesh
# add_hparams
# flush()
# close()

[用例](./__summarywriter.py)

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

x = torch.arange(-5, 5, 0.1).view(-1, 1)
y = -5 * x + 0.1 * torch.randn(x.size())

model = torch.nn.Linear(1, 1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

def train_model(iter):
    for epoch in range(iter):
        y1 = model(x)
        loss = criterion(y1, y)
        writer.add_scalar("Loss/train", loss, epoch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

train_model(10)
writer.close()

# 之后在命令行调用 tensorboard --logdir=runs 即可在浏览器中显示

In [ ]:
# CLASS torch.utils.tensorboard.wrier.SummaryWriter
# 将记录直接写入位于log_dir文件夹中的event files

In [ ]:
# __init__(log_dir=None, comment="", purge_step=None, max_queue=10,
# flush_secs=120, file_name_suffix="")
# 初始化SummaryWriter实例

# 参数:
# 1. log_dir(str): 存档目录, 默认为runs/当前日期与时间-主机名(记录名)
# 如 May07_11-02-39_DESKTOP-ABCDEFG
# 通过层次化文件夹, 如./runs/exp1, runs/exp2, 可以用来比较不同的experiment
# 2. comment(str): 将event files放在runs/默认log_dir+comment/里, 如果log_dir被指定, 则该参数无效
# 3. purge_step(int): 当logging在第T+X步崩溃并回到第T步重启时,
# 所有大于等于T步的event都会被清除
# 4. max_queue(int): 在add类型方法调用flush强制将event写入磁盘前, 容许的最多的pending events数量
# 5. flush_secs(int): 自动flush event的周期, 默认是120秒
# 6. filename_suffix(str): 添加后缀名到所有log_dir文件夹中的event files上 

In [ ]:
# 例:

from torch.utils.tensorboard import SummaryWriter

# 创建一个默认SummaryWriter
writer = SummaryWriter()
# folder location: runs/May04_22_14_54_s-MacBook-Pro.local/

# 创建指定log_dir的SummaryWriter
writer = SummaryWriter("my_experiment")
# folder location: my_experiment

# 创建指定comment后缀的SummaryWriter
writer = SummaryWriter(comment="LR_0.1_BATCH_16")
# folder location: runs/May04_22-14-54_s-MacBook-
# Pro.LocalLR_0.1_BATCH_16/

In [ ]:
# add_scalar(tag, scalar_value, global_step=None, walltime=None, new_style=False,
# double_precision=False)
# 添加标量数据至summary中

# 参数:
# tag(str): 数据标识
# tag可用于显示的分组, 形如"A/B"的tag, 其分组为A, 组内名称为B
# scalar_value(float or string/blobname): 指定要存储的变量
# global_step(int): 标识当前全局轮次
# walltime(float): 可选参数, 默认为time.time(), 指定记录标量时的时间戳, 可用于对齐不同操作
# new_style(boolean): 决定使用新样式(tensor field)或老样式(simple_value field)
# 新样式加载数据速度更快

In [7]:
# 例:

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
x = range(100)
for i in x:
    writer.add_scalar("y=2x", i * 2, i)
writer.close()

In [ ]:
# add_scalars(main_tag, tag_scalar_dict, global_step=None, walltime=None)
# 批量添加标量数据至summary

# 参数
# main_tag(str): 父tag名
# tag_scalar_dict(dict): tag: value形式的键值对的dict
# global_step(int): 标识当前全局轮次
# walltime(float): 可选参数, 默认为time.time(), 指定记录标量时的时间戳, 可用于对齐不同操作

In [10]:
# 例:

import numpy as np
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
r = 5
for i in range(100):
    writer.add_scalars("run_14h", {"xsinx": i*np.sin(i/r),
                                   "xcosx": i*np.cos(i/r),
                                   "tanx": np.tan(i/r)}, i)
writer.close()

In [ ]:
# add_histogram(tag, values, global_step=None, bins="tensorflow", walltime=None,
# max_bins=None)
# 记录张量的分布直方图, 可用于动态追踪模型参数, 激活值或自定义数据的分布变化

# 参数:
# tag(str): 数据标识
# values(torch.Tensor, numpy.ndarry, or string/blobname): 直方图纵坐标对应的变量
# global_step(int): 标识当前全局轮次
# bins(str or int): {"tensorflow", "auto", "fd", ...}中的其中一个, 决定分桶策略
# 可在https://docs.scipy.org/doc/numpy/reference/generated/numpy.histogram.html找到其他选项
# 为int类型如50时, 为自定义分桶数量
# walltime(float): 可选参数, 默认为time.time(), 指定记录标量时的时间戳, 可用于对齐不同操作
# max_bins: 最大分桶数, 仅当bins="tensorflow"时生效

In [13]:
# 例:

from torch.utils.tensorboard import SummaryWriter
import numpy as np
writer = SummaryWriter()
for i in range(10):
    # 生成随机数据: 1000个[0, 1)上均匀分布的数
    x = np.random.random(1000)

    # 记录直方图
    #   - tag: "distribution centers"
    #   - values: x + i (数据, 每次循环右移i, 显示动态变化)
    #   - global_step: i (横轴位置)
    writer.add_histogram("distribution centers", x + i, i)

# 关闭写入器, 确保数据写入磁盘
writer.close()

In [14]:
# add_image(tag, img_tensor, global_step=None, walltime=None,
# dataformats="CHW")
# 将图像数据写入summary, 需要pillow包

# 参数:
# tag(str): 数据标识
# img_tensor(torch.Tensor, numpy.ndarray, or string/blobname): 图像数据
# global_step(int): 标识当前全局轮次
# walltime(float): 可选参数, 默认为time.time(), 指定记录标量时的时间戳, 可用于对齐不同操作
# dataformats(str): 图像数据的格式, 如CHW(默认), HWC, HW, WH等
# 其中C=channels, H=height, W=width

In [2]:
# 例:

from torch.utils.tensorboard import SummaryWriter
import numpy as np
img = np.zeros((3, 100, 100))
img[0] = np.arange(0, 10000).reshape(100, 100) / 10000
img[1] = 1 - np.arange(0, 10000).reshape(100, 100) / 10000

img_HWC = np.zeros((100, 100, 3))
img_HWC[:, :, 0] = np.arange(0, 10000).reshape(100, 100) / 10000
img_HWC[:, :, 1] = 1 - np.arange(0, 10000).reshape(100, 100) / 10000

writer = SummaryWriter()
writer.add_image("my_image", img, 0)

# 如果不是默认图像格式, 则需要设置图像格式
writer.add_image("my_image_HWC", img_HWC, 0, dataformats="HWC")
writer.close()

In [3]:
# add_images(tag,, img_tensor, global_step=None, walltime=None,
# dataformats = 'NCHW')
# 批量写入image到summary, 需要pillow包

# 参数:
# tag(str): 数据标识
# img_tensor(torch.Tensorm, numpy.ndarry, or string/blobname): 图像数据
# global_step(int): 记录全局轮次
# walltime(float): 可选参数, 默认为time.time(), 指定记录标量时的时间戳, 可用于对齐不同操作
# dataformats(str): 图像数据格式, 默认NCHW, 可以是NHWC, CHW, HWC, HW, WH等等
# N: 图像数量, C: 通道数量, H: 高度, W: 宽度


In [ ]:
# 例:

from torch.utils.tensorboard import SummaryWriter
import numpy as np

img_batch = np.zeros((16, 3, 100, 100))
for i in range(16):
    img_batch[i, 0] = np.arange(0, 10000).reshape(100, 100) / 10000 / 16 * i
    img_batch[i, 1] = (1 - np.arange(0, 10000).reshape(100, 100) / 10000) / 16 * i

writer = SummaryWriter()
writer.add_images('my_image_batch', img_batch, 0)
writer.close()

In [5]:
# add_figure(tag, figure, global_step=None, close=True, walltime=None)
# 将matplotlib的figure对象转化为image, 再添加到summary中
# 需要matplotlib包

# 参数:
# tag(str): 数据标识
# figure(Union[Figure, list['Figure']]): figure或者figure列表
# global_step(Optional[int]): 全局轮次
# close(bool): 是否自动关闭figure
# walltime(Optional[float]): 可选参数, 默认为time.time(), 指定记录标量时的时间戳, 可用于对齐不同操作


In [6]:
# add_video(tag, vid_tensor, global_step=None, fps=4, walltime=None)
# 添加视频到summary
# 需要moviepy包

In [7]:
# add_audio(tag, snd_tensor, global_step=None, sample_rate=44100,
# walltime=None)
# 添加音频到summary

In [ ]:
# add_text(tag, text_string, global_step=None, walltime=None)
# 添加文本到summary

# 参数:
# tag(str): 数据标识
# text_string(str): 要保存的文本
# global_step(int): 全局步数
# walltime(Optional[float]): 可选参数, 默认为time.time(), 
# 指定记录标量时的时间戳, 可用于对齐不同操作

In [10]:
# 例:

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

writer.add_text('lstm', 'This is an lstm', 0)
writer.add_text('rnn', 'This is an rnn', 10)
writer.close()

In [ ]:
# add_graph(model, input_to_model=None, verbose=False,
# use_strict_trace=True)
# 添加graph数据到summary
# graph指的是模型架构, 而不是图神经网络

# 参数:
# model(torch.nn.Module): 需要绘制的模型
# inpt_to_model(torch.Tensor or list of torch.Tensor):
# 待输入的变量或元组
# verbose(bool): 是否在控制台打印图结构
# use_strict_trace(bool): 是否向torch.jit.trace传递关键字参数strict
# 值为false代表让tracer记录容器类型(list, dict)

In [ ]:
# add_embedding(mat, metadata=None, label_img=None, global_step=None,
# tag='default', metadata_header=None)
# 将embedding projector添加到summary

# mat(torch.Tensor or numpy.ndarray): 矩阵, 每行是数据点的特征向量
# metadata(list): 标签的list, 其中每个元素会被转换成string
# label_img(torch.Tensor): 与每个数据点相匹配的images
# global_step(int): 全局轮次
# tag(str): embedding的名称
# metadata_header(list): 给具有多列的metadata添加headers, 如果给定
# 则每个metadata必须具有与headers相匹配的values
# Shape:e
#   mat: (N, D), N是数据数量, D是特征维数
#   label_img: (N, C, H, W)

In [2]:
# 例:

import keyword
import torch
from torch.utils.tensorboard import SummaryWriter

meta = []
while len(meta) < 100:
    meta = meta + keyword.kwlist
meta = meta[:100]

for i, v in enumerate(meta):
    meta[i] = v + str(i)

label_img = torch.rand(100, 3, 10, 32)
for i in range(100):
    label_img[i] *= i / 100.0

writer = SummaryWriter()
writer.add_embedding(torch.randn(100
    , 5), metadata=meta,
    label_img=label_img)
writer.add_embedding(torch.randn(100
    , 5), label_img=label_img)
writer.add_embedding(torch.randn(100
    , 5), metadata=meta)
writer.close()

In [3]:
# add_pr_curve(tag, labels, predictions, global_step=None,
# num_thresholds=127, weights=None, walltime=None)
# 添加precision-recall曲线, TensorBoard UI可以交互式调整threshold

# 参数:
# tag(str): 数据标识
# labels(torch.Tensor, numpy.ndarray, or string/blobname): 已知正确答案
# predictions(torch.Tensor, numpy.ndarray, or string/blobname): 预测该类为正类的概率(0到1)
# global_step(int): 记录全局轮次
# num_thresholds(int): thresholds采样点数
# walltime(float): 可选参数, 默认为time.time(), 
# 指定记录标量时的时间戳, 可用于对齐不同操作

In [4]:
# 例:

from torch.utils.tensorboard import SummaryWriter
import numpy as np

labels = np.random.randint(2, size=100)
predictions = np.random.rand(100)
writer = SummaryWriter()
writer.add_pr_curve('pr_curve', labels, predictions, 0)
writer.close()

In [6]:
# add_hparams(hparam_dict, metric_dict, hparam_domain_discrete=None,
# run_name=None, global_step=None)
# 添加待比较的超参数集

# 参数:
# hparam_dict(dict): 超参数字典, value可以是bool, string, float, int, 或者None
# metric_dict(dict): 度量字典, key必须唯一
# 否则通过add_Scalar添加的值也会被hparam调用
# hparam_domain_discrete: Optional[Dict[str, List[Any]]]:
# 包含所有超参数名称和其可取的离散值列表
# run_name(str): run的名称, None时默认为当前时间戳
# global_step(int): 记录全局轮次

In [7]:
from torch.utils.tensorboard import SummaryWriter
with SummaryWriter() as w:
    for i in range(5):
        w.add_hparams({
            'lr': 0.1*i, 
            'bsize': i
        }, {
            'hparam/accuracy': 10*i,
            'hparam/loss': 10*i
        })

In [8]:
# flush()
# 将所有event files写入磁盘

In [ ]:
# close()
# 关闭summarywriter, 且在关闭前会调用flush()